## Table of Contents
1. [Scarce scrap](#scarce_scrap)
2. [Steel production dependent scrap](#dependent_scrap)
3. [Steel production independent scrap](#independent_scrap)

In [ ]:
import datetime
import os
import pathlib

import dotenv
import plotly.graph_objects as go
import plotly.express as px

from usskscadaocclient import ScadaConnector
from scrap_core.home_scrap_estimation.common import SCARCE_SCRAP, DEPENDENT_SCRAP, INDEPENDENT_SCRAP, \
    estimate_dependent_home_scrap, DEFAULT_SCRAP_TYPE_MAP_NAME, DEFAULT_CONFIG_ENV, get_home_scrap_and_steel_production_data

In [ ]:
dotenv.load_dotenv(DEFAULT_CONFIG_ENV)

print(os.environ.get("USE_DB"))

In [ ]:
workdir = pathlib.Path(r"C:\Users\dec4288\OneDrive - United States Steel\ProjectData\scrap\scrap_supplies")

dt_from = datetime.date(2020, 1, 1)
dt_to = datetime.date(2023, 8, 1)

SCADA_DB = {
    "host": os.environ.get("SCADA_DB_HOST"),
    "port": os.environ.get("SCADA_DB_PORT"),
    "database": os.environ.get("SCADA_DB_SID"),
    "username": os.environ.get("SCADA_DB_USER"),
    "password": os.environ.get("SCADA_DB_PASSWORD"),
}

In [ ]:
df_production = get_home_scrap_and_steel_production_data(workdir, DEFAULT_SCRAP_TYPE_MAP_NAME, ScadaConnector(conf=SCADA_DB), dt_from, dt_to)

## Scarce scrap  <a name="scarce_scrap"></a>
- HDS, HSK, HSR Cr, HSR, BPIT, SRB
- expected production of scarce scrap is 0

In [ ]:
df_production.head()

In [ ]:
px.line(df_production.sort_index()[SCARCE_SCRAP])

## Steel production dependent scrap  <a name="dependent_scrap"></a>
- HS, HSZ, DSI, 1PIT, HSB, HST
- assumed linear relationship between steel and scrap production

In [ ]:
estimate_cols = []

for scrap_type in DEPENDENT_SCRAP:
    df_production[scrap_type + "_est"] = df_production["final_steel_w"].apply(lambda steel_w: estimate_dependent_home_scrap(scrap_type, steel_w))
    estimate_cols.append(scrap_type + "_est")

px.line(df_production[DEPENDENT_SCRAP + estimate_cols])

In [ ]:
x = df_production["final_steel_w"]

fig = go.Figure()

for scrap_type in DEPENDENT_SCRAP:
    y = df_production[scrap_type]

    fig.add_trace(go.Scatter(x=x, y=y, mode="markers", name=scrap_type))

    fig.add_trace(
        go.Scatter(
            x=x, y=x.apply(lambda steel_w: estimate_dependent_home_scrap(scrap_type, steel_w)), mode="lines+markers", name=scrap_type
        ),
    )

fig.show()

## Steel production independent scrap  <a name="independent_scrap"></a>
- 2PIT, HSD, PIG IRON
- scrap production does not depend on steel production

In [ ]:
px.scatter(df_production, x="final_steel_w", y=INDEPENDENT_SCRAP)

In [ ]:
df_estimates = df_production[INDEPENDENT_SCRAP].rolling(12).median()

df_tmp = df_production[INDEPENDENT_SCRAP].join(df_estimates, lsuffix="_obs", rsuffix="_est").reset_index()

px.line(df_tmp[[col for col in df_tmp.columns if col.endswith(("obs", "est"))]])